<a href="https://colab.research.google.com/github/pk-790324/Tensor-Flow-Series/blob/main/CNN/malaria_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D,Flatten,MaxPooling2D,Dense

In [ ]:
datasets,dataset_info=tfds.load('malaria',with_info=True,as_supervised=True,shuffle_files=True,split=['train'])

In [ ]:
datasets

In [ ]:
dataset_info

In [ ]:
for data in datasets[0].take(2):
  print(data)


In [ ]:
#creating a function for train,test,validation
def train_test_val_split(datasets,train_ratio,test_ratio,val_ratio):
  data_len=len(datasets)
  train_data=datasets.take(int(train_ratio*data_len))
  test_val_data=datasets.skip(int(train_ratio*data_len))
  test_data=test_val_data.take(int(test_ratio*len(test_val_data)))
  val_data=test_val_data.skip(int(test_ratio*len(test_val_data)))
  return train_data,test_data,val_data

In [ ]:
train_data,test_data,val_data=train_test_val_split(datasets[0],train_ratio=0.7,test_ratio=0.7,val_ratio=0.3)

In [ ]:
len(datasets[0]),len(train_data),len(test_data),len(val_data)

In [ ]:
print(list(train_data.take(2).as_numpy_iterator()))

# Data Visualization

In [ ]:
for i,(image,label) in enumerate(train_data.take(16)):
  plt.subplot(4,4,i+1)
  plt.imshow(image)
  plt.title(dataset_info.features['label'].int2str(label))
  plt.axis('off')

# Data Preprocessing

In [ ]:
def img_resize_scale(image,label):
  return tf.image.resize(image,(224,224))/255.0,label

In [ ]:
train_data=train_data.map(img_resize_scale)
test_data=test_data.map(img_resize_scale)
val_data=val_data.map(img_resize_scale)

In [ ]:
for image,label in train_data.take(1):
  print(image,label)

In [ ]:
train_data=train_data.shuffle(buffer_size=100,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)
test_data=test_data.shuffle(buffer_size=100,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)
val_data=val_data.shuffle(buffer_size=100,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
model=Sequential()
model.add(Conv2D(6,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu',input_shape=(224,224,3)))
tf.keras.layers.BatchNormalization()
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(16,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu'))
tf.keras.layers.BatchNormalization()
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
tf.keras.layers.BatchNormalization()
model.add(Dense(10,activation='relu'))
tf.keras.layers.BatchNormalization()
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
#compile the model
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
loss=tf.keras.losses.BinaryCrossentropy()
metrics=['accuracy']
model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [ ]:
history=model.fit(train_data,epochs=5,validation_data=val_data)

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot()
plt.xlabel('no of epochs')
plt.ylabel('accuracy')
plt.title('train and validation data curve')

# model evaluation and prediction

In [ ]:
#model evaluation
model.evaluate(test_data)


In [ ]:
#model prediction
model.predict(test_data.take(1))[0][0]

In [ ]:
def parasite_or_not(x):
  if x<0.5:
    return str('P')
  else:
    return str('U')

In [ ]:
parasite_or_not(model.predict(test_data.take(1))[0][0])

In [ ]:
for i,(image,label) in enumerate(test_data.take(25)):
  plt.subplot(5,5,i+1)
  plt.imshow(image[0])
  plt.title(str(parasite_or_not(label.numpy()[0]))+ ':' + str (parasite_or_not(model.predict(image)[0][0])))
  plt.axis('off')

#Create functional api

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

In [ ]:
funct_input=Input(shape=(224,224,3))
x=Conv2D(6,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu',)(funct_input)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

x=Conv2D(16,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu',)(funct_input)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

x=Flatten()(x)

x=Dense(100,activation='relu')(x)
x=BatchNormalization()(x)
x=Dense(100,activation='relu')(x)
x=BatchNormalization()(x)
output=Dense(1,activation='sigmoid')(x)


In [ ]:
model_function_api=Model(inputs=funct_input,outputs=[output])

In [ ]:
model_function_api.summary()

In [ ]:
#compile the model
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
loss=tf.keras.losses.BinaryCrossentropy()
metrics=['accuracy']
model_function_api.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [ ]:
history_function_api=model_function_api.fit(train_data,epochs=5,validation_data=val_data)

In [ ]:
import pandas as pd
pd.DataFrame(history_function_api.history).plot()
plt.xlabel('no of epochs')
plt.ylabel('accuracy')
plt.title('train and validation data curve')

#Model evaluatin and prediction

In [ ]:
model_function_api.evaluate(test_data)

In [ ]:
#model prediction
model.predict(test_data.take(1))[0][0]

In [ ]:
def parasite_or_not(x):
  if x<0.5:
    return str('P')
  else:
    return str('U')

In [ ]:
for i,(image,label) in enumerate(test_data.take(25)):
  plt.subplot(5,5,i+1)
  plt.imshow(image[0])
  plt.title(str(parasite_or_not(label.numpy()[0]))+ ':' + str (parasite_or_not(model.predict(image)[0][0])))
  plt.axis('off')